In [ ]:
!git clone https://github.com/rmnigm/qber-forecasting.git
!pip install wandb
!pip install pytorch_lightning torchmetrics

In [5]:
!cp -r qber-forecasting/deep_qber deep_qber

In [2]:
import os
import random
import sys

import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import mean_squared_error, mean_absolute_percentage_error 

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

In [6]:
from deep_qber import seed_everything, setup_dataset
from deep_qber import TorchTSDataset, ModelInterfaceTS, ModuleTS

In [7]:
# optional
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
seed_everything(123456)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
pulses_stats_file_path = "/content/qber-forecasting/datasets/fr_gains.csv"
dataframe = pd.read_csv(pulses_stats_file_path,
                        usecols=[0, 1, 2, 3, 4, 5, 6],
                        engine='python'
                        )
dataset = dataframe.values.astype('float32')
dataset = dataset[:100000]

In [10]:
# scaler = QuantileTransformer(n_quantiles=20, output_distribution="normal")
# scaler = StandardScaler()
config = {
    "learning_rate": 1e-5,
    "look_back": 20,
    "batch_size": 256,
    "epochs": 5,
    "loss": "MSE",
    "scaler": None,
    "model": "LSTM-LeakyReLu-Dense"
}
train_size = 0.8
loss = nn.MSELoss()
scaler = None

train_loader, test_loader = setup_dataset(dataset,
                                          config["look_back"],
                                          train_size,
                                          config["scaler"],
                                          config["batch_size"]
                                          )

Training set size = 80000, testing set size = 20000


In [11]:
class extract_tensor(nn.Module):
    def forward(self,x):
        # Output shape (batch, features, hidden)
        tensor, _ = x
        # Reshape shape (batch, hidden)
        return tensor[:, -1, :]

input_size = 7
output_size = 1
hidden_size = 128

model = nn.Sequential(
    nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True),
    extract_tensor(),
    nn.LeakyReLU(),
    nn.Linear(hidden_size * 2, output_size),
    ).to(device)

In [12]:
def run_experiment(train_loader, test_loader, model, loss, config):
    with wandb.init(project="qber-forecasting",
                    entity="rmnigm",
                    settings=wandb.Settings(start_method="thread"),
                    config=config,
                    ) as run:
        wandb_logger = WandbLogger(log_model='all')
        checkpoint_callback = ModelCheckpoint(monitor="Validation MAPE", mode="max")

        epochs = config["epochs"]

        model_interface = ModelInterfaceTS(model)
        module = ModuleTS(model_interface, loss, lr=config["learning_rate"])

        trainer = pl.Trainer(logger=wandb_logger,
                            callbacks=[checkpoint_callback],
                            # accelerator="gpu",
                            max_epochs=epochs,
                            )
        
        trainer.fit(module, train_loader, test_loader)

        run.finish()

In [ ]:
run_experiment(train_loader, test_loader, model, loss, config)